# LangGraph tutorial 🤖🤖🤖
By the end of this tutorial, you will create a simple LangGraph workflow like this visualization.

## Getting Started

In [2]:
! pip install --quiet langchain langchain-ollama langgraph chromadb beautifulsoup4 langchain-community pandas

### Get Llama 3.1 model

If you don't have `llama3.1` yet, follow this url to download it locally: https://github.com/ollama/ollama

## Putting them All Together

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

embd = OllamaEmbeddings(model="llama3.1")

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embd,
)

# Create retriever
retriever = vectorstore.as_retriever()

In [ ]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in doc_splits], columns=["text"])
df

,text
0,LLM Powered Autonomous Agents | Lil'Log\n\n\n\...
1,LLM Powered Autonomous Agents\n \nDate: Jun...
2,Component Two: Memory\n\nTypes of Memory\n\nMa...
3,Building agents with LLM (large language model...
4,"well-written copies, stories, essays and progr..."
...,...
725,[21] Greshake et al. “Compromising Real-World ...
726,[22] Jain et al. “Baseline Defenses for Advers...
727,[24] Wei & Zou. “EDA: Easy data augmentation t...
728,Nlp\nLanguage-Model\nSafety\nAdversarial Attac...


In [23]:
question = "What are some techniques for prompt engineering?"
documents = retriever.invoke(question)
documents = [d.page_content for d in documents]
documents

['Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.',
 'Level-2 examines the ability to retrieve the API. The model needs to search for possible APIs that may solve the user’s requirement and learn how to use them by reading documentation.',
 'Prompt Engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes without updating the model weights. It is an empirical science and the effect of prompt engineering methods can vary a lot among models, thus requiring heavy experimentation and heuristics.\nThis post only focuses on prompt engineering for autoregressive language models, so nothing with Cloze tests, image generation or multimodality models. At its core, the goal of prompt engineering is about alignment and model steerability. Check my previous post on controllable text generation.',
 'Experiments on AlfWorld 

In [53]:

from typing_extensions import TypedDict
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate

# Get the model
model = ChatOllama(model="llama3.1")

# Declare the State
class State(TypedDict):
    question: str
    documents: list
    genai_response: str
    is_answer_correct: str # pick from ["yes", "no"]


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    state["documents"] = [d.page_content for d in documents]
    return state


def genai_executes(state: State):
    # `state` contains question
    
    question = state["question"]
    template = """You are a helpful assistant that answers questions.
    Here are some documents to help you answer the question:
    {documents}
    Please write the answer in 5 sentences
    Here is the question:\n {question} """

    prompt = ChatPromptTemplate.from_template(template)

    print("==================== Original Prompt ====================")
    import pprint
    pprint.pprint(template)
    # 🔍 Print the final prompt with variables filled in
    formatted_prompt = prompt.format(question=question, documents=documents)

    print("==================== Prompt Sent to Model ====================")
    import pprint
    pprint.pprint(formatted_prompt)
    print("==============================================================")


    print("---GENAI---")
    genai_chain = prompt | model | StrOutputParser()
    response = genai_chain.invoke({"question": question, "documents": state["documents"]})
    
    # Add response to state dict
    state["genai_response"] = response
    return state

class CorrectnessChecker(BaseModel):
        """Check if answer is correct or not"""
        is_correct: Literal["yes", "no"] = Field(
            ...,
            description="Return 'yes' if the answer is correct. Otherwise, return 'no'"
        ) 

def correctness_checker_executes(state: State):
    # `state` contains question and genai_response
    print("==============================================================")
    print("---CHECK-CORRECT---")
    question = state["question"]
    genai_response = state["genai_response"]

    # preamble
    system_prompt = """You are a helpful assistant that determine the correctness of
        the GenAI response from a question. Here is the question: \n {question}\n
        Here is the GenAI response: \n {genai_response}"""
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{question}"),
        ]
    )
    structured_model = model.with_structured_output(CorrectnessChecker)
    
    correctness_checker_chain = prompt | structured_model 
    state["is_answer_correct"] = correctness_checker_chain.invoke({"question": question, "genai_response": genai_response}).is_correct
    return state

def is_response_correct(state: State):
    return True if state["is_answer_correct"] == "yes" else False


def create_workflow() -> StateGraph:
    workflow = StateGraph(State)

    # Add node
    workflow.add_node("retrieve", retrieve)
    workflow.add_node("genai", genai_executes)
    workflow.add_node("correctness_checker", correctness_checker_executes)

    # Add edges and conditional edges
    workflow.add_edge(START, "retrieve")
    workflow.add_edge("retrieve", "genai")
    workflow.add_edge("genai", "correctness_checker")
    workflow.add_conditional_edges(
        "correctness_checker",
        is_response_correct,
        {
            False: "genai",
            True: END
        }
    )
    return workflow


In [54]:
workflow = create_workflow()
graph = workflow.compile()


# Run
import pprint
inputs = {"question": "What are some techniques for prompt engineering?"}
for output in graph.stream(inputs, {"recursion_limit": 100}):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        if key == "retrieve":
            pprint.pprint("Retrieved Documents:")
            pprint.pprint(value["documents"], indent=2, width=80, depth=None)
        elif key == "genai":
            pprint.pprint("GenAI Response:")
            pprint.pprint(value["genai_response"], indent=2, width=80, depth=None)
        elif key == "correctness_checker":
            pprint.pprint("Correctness Check Result:")
            pprint.pprint(value["is_answer_correct"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
print("==============================================================")
pprint.pprint("Final Generation:")
pprint.pprint(value["genai_response"])

---RETRIEVE---
"Node 'retrieve':"
'Retrieved Documents:'
[ 'Short-term memory: I would consider all the in-context learning (See Prompt '
  'Engineering) as utilizing short-term memory of the model to learn.',
  'Level-2 examines the ability to retrieve the API. The model needs to search '
  'for possible APIs that may solve the user’s requirement and learn how to '
  'use them by reading documentation.',
  'Prompt Engineering, also known as In-Context Prompting, refers to methods '
  'for how to communicate with LLM to steer its behavior for desired outcomes '
  'without updating the model weights. It is an empirical science and the '
  'effect of prompt engineering methods can vary a lot among models, thus '
  'requiring heavy experimentation and heuristics.\n'
  'This post only focuses on prompt engineering for autoregressive language '
  'models, so nothing with Cloze tests, image generation or multimodality '
  'models. At its core, the goal of prompt engineering is about alignmen